# Marginalization

In this notebook, we will load the data from after the second fit and perform the marginalization.

I need to get the data import working. The cells will have to be broken up more. I need to check for astropy units.

In [ ]:
# Imports
import numpy as np
import os
import time
import sys
import matplotlib.pyplot as plt
%matplotlib inline
import astropy.units as u
from astropy.constants import G

os.chdir('..')
from config import CONFIG_INI
import margmodule as marg

## Load the data

Load the results into the notebook and make sure the data looks right.

In [ ]:
# ... load data here

## Reformat all arrays with just positive values

Why?

In [ ]:
a = (np.sort(sys_evidenceAIC))[::-1]
print('\nTOP 10 SYSTEMATIC MODELS')

#TODO: What is getting printed here?
print(a[:10])
#TODO: What is getting printedx here?
print(sys_evidenceAIC)

# REFORMAT all arrays with just positive values
pos = np.where(sys_evidenceAIC > -500)   #TODO: change hard coded number?
if len(pos) == 0:
    pos = -1
npos = len(pos[0])   # NOT-REUSED
#TODO: What is getting printed here?
print('POS positions = {}'.format(pos))

count_AIC = sys_evidenceAIC[pos]

count_depth = sys_depth[pos]
count_depth_err = sys_depth_err[pos]

count_epoch = sys_epoch[pos]
count_epoch_err = sys_epoch_err[pos]

count_residuals = sys_residuals[pos]
count_date = sys_date[pos]
count_flux = sys_flux[pos]
count_flux_err = sys_flux_err[pos]
count_phase = sys_phase[pos]
count_model_y = sys_model[pos]
count_model_x = sys_model_phase[pos]

## More calculations

In [ ]:
beta = np.min(count_AIC)
w_q = (np.exp(count_AIC - beta)) / np.sum(np.exp(count_AIC - beta))

n01 = np.where(w_q >= 0.05)
print('{} models have a weight over 0.1. -> Models:'.format(len(n01), n01, w_q[n01]))
print('Most likely model is number {} at w_q={}'.format(np.argmax(w_q), np.max(w_q)))

best_sys = np.max(w_q)

rl_sdnr = np.zeros(len(w_q))
for i in range(len(w_q)):
    rl_sdnr[i] = (np.std(count_residuals[:, i]) / np.sqrt(2)) * 1e6
best_sys = np.argmin(rl_sdnr)

print('SDNR best model = {}'.format(np.std(count_residuals[:, best_sys]) / np.sqrt(2) * 1e6))
print('SDNR best = {} for model {}'.format(np.min(rl_sdnr), np.argmin(rl_sdnr)))

## Plots

In [ ]:
plt.figure(3)
plt.subplot(3,1,1)
plt.plot(w_q)
plt.title('w_q')
plt.subplot(3,1,2)
plt.plot(rl_sdnr)
plt.title('rl_sdnr')
plt.subplot(3,1,3)
plt.errorbar(np.arange(1, len(count_depth)+1), count_depth, yerr=count_depth_err, fmt='.')
plt.title('count_depth')
plt.draw()
plt.pause(0.05)

plt.figure(4)
plt.subplot(3, 1, 1)
plt.scatter(sys_phase[0,:], sys_flux[0,:])
plt.ylim(np.min(sys_flux[0,:]) - 0.001, np.max(sys_flux[0,:]) + 0.001)
plt.xlabel('sys_phase')
plt.ylabel('sys_flux')

plt.subplot(3,1,2)
plt.scatter(count_phase[best_sys,:], count_flux[best_sys,:])
plt.plot(count_model_x[best_sys,:], count_model_y[best_sys,:])
plt.ylim(np.min(count_flux[0,:]) - 0.001, np.max(count_flux[0,:]) + 0.001)
plt.xlabel('count_phase')
plt.ylabel('count_flux')

plt.subplot(3,1,3)
plt.errorbar(count_phase[best_sys,:], count_residuals[best_sys,:], yerr=count_flux_err[best_sys,:], fmt='.')
plt.ylim(-1000, 1000)
plt.xlabel('count_phase')
plt.ylabel('count_residuals')
plt.hlines(0.0, xmin=np.min(count_phase[best_sys,:]), xmax=np.max(count_phase[best_sys,:]), colors='r', linestyles='dashed')
#plt.hlines(0.0 - (rl_sdnr[best_sys] * cut_down), xmin=np.min(count_phase[best_sys,:]), xmax=np.max(count_phase[best_sys,:]), colors='r', linestyles='dotted')
#plt.hlines(0.0 + (rl_sdnr[best_sys] * cut_down), xmin=np.min(count_phase[best_sys,:]), xmax=np.max(count_phase[best_sys,:]), colors='r', linestyles='dotted')
plt.draw()
plt.show()

## Perform the marginalization

In [ ]:
### Radius ratio
marg_rl, marg_rl_err = marg.marginalization(count_depth, count_depth_err, w_q)
print('Rp/R* = {} +/- {}'.format(marg_rl, marg_rl_err))

In [ ]:
### Center of transit time
marg_epoch, marg_epoch_err = marg.marginalization(count_epoch, count_epoch_err, w_q)
print('Epoch = {} +/- {}'.format(marg_epoch, marg_epoch_err))

In [ ]:
### Inclination in radians
marg_inclin_rad, marg_inclin_rad_err = marg.marginalization(sys_params[:, 3], sys_params_err[:, 3], w_q)
print('inc (rads) = {} +/- {}'.format(marg_inclin_rad, marg_inclin_rad_err))

In [ ]:
### Inclination in degrees
conv1 = sys_params[:, 3] / (2 * np.pi / 360)
conv2 = sys_params_err[:, 3] / (2 * np.pi / 360)
marg_inclin_deg, marg_inclin_deg_err = marg.marginalization(conv1, conv2, w_q)
print('inc (deg) = {} +/- {}'.format(marg_inclin_deg, marg_inclin_deg_err))

In [ ]:
### MsMpR
marg_msmpr, marg_msmpr_err = marg.marginalization(sys_params[:, 4], sys_params_err[:, 4], w_q)
print('MsMpR = {} +/- {}'.format(marg_msmpr, marg_msmpr_err))

marg_aors = constant1 * (marg_msmpr ** 0.333)
marg_aors_err = constant1 * (marg_msmpr_err ** 0.3333) / marg_aors
print('a/R* = {} +/- {}'.format(marg_aors, marg_aors_err))